# Шаг 1. Парсинг данных

### Парсинг данных индексов IMOEX, MOEXMM ###

Цены на золото исторически имеют большую зависимость от фондового рынка, индикатором которого служит индекс капитализации фондового рынка IMOEX, и добычи металлов, индикатором которой служит индекс MOEXMM. Поэтому для анализа были выбраны именно эти индексы

Код основан на MOEX API с использованием https://iss.moex.com/iss/reference/

In [1]:
# импорт библиотек для парсинга
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore")

In [2]:
# устанавливаем дату, с которой начнем собирать данные и которой закончим сбор для данных MOEX

start_date = '2013-09-01'
end_date = '2025-01-01'

In [3]:
# для индекса IMOEX

def get_imoex(start_date, end_date, page_size=100):
    
    '''Функция зависит от переменных: start_date - дата начала сбора данных, end_date - дата окончания сбора данных,
    page_size - количество данных на одной странице, равное 100. Внутри функции создается список imoex_data с хранением информации, счетчик
    count отсчитывает данные, находящиеся на одной странице и служит индикатором перехода на другую. '''

    imoex_data = []
    count = 0
    while True:
        # используя MOEX ISS получаем url сайта
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/IMOEX.json?from={start_date}&till={end_date}&start={count}&limit={page_size}'
        try:
            response = requests.get(url)  # получаем доступ к сайту
            data = response.json()  # получаем данные из API и переводим из JSON в python формат
            history_data = data['history']['data']
            headings = data['history']['columns']
            if history_data != None:
                imoex_data.extend(history_data)
                count += len(
                    history_data)  # увеличиваем счётчик, для того чтобы на следующей странице начать с этого номера
        except:
            return pd.DataFrame(imoex_data, columns=headings)  # если данные закончились возвращаем imoex_data

        if len(history_data) < page_size:  # останавливаем бесконечный цикл
            break
    return pd.DataFrame(imoex_data, columns=headings)


page_size = 100
IMOEX_df_before_EDA = get_imoex(start_date, end_date, page_size=page_size)
print(IMOEX_df_before_EDA.shape)
IMOEX_df_before_EDA.head(10)

(2845, 18)


,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,2013-09-02,Индекс МосБиржи,Индекс МосБиржи,1367.53,1364.75,1373.73,1364.27,1.274407e+10,NaN,NaN,2,6.096417e+12,RUB,NaN,3,None
1,SNDX,IMOEX,2013-09-03,Индекс МосБиржи,Индекс МосБиржи,1373.82,1366.99,1384.18,1366.99,2.216425e+10,NaN,NaN,2,6.124461e+12,RUB,NaN,3,None
2,SNDX,IMOEX,2013-09-04,Индекс МосБиржи,Индекс МосБиржи,1375.66,1372.98,1376.42,1365.21,1.692969e+10,NaN,NaN,2,6.132675e+12,RUB,NaN,3,None
3,SNDX,IMOEX,2013-09-05,Индекс МосБиржи,Индекс МосБиржи,1422.40,1375.66,1422.40,1373.29,3.763559e+10,NaN,NaN,2,6.341025e+12,RUB,NaN,3,None
4,SNDX,IMOEX,2013-09-06,Индекс МосБиржи,Индекс МосБиржи,1423.40,1421.86,1448.75,1418.80,5.218912e+10,NaN,NaN,2,6.345500e+12,RUB,NaN,3,None
5,SNDX,IMOEX,2013-09-09,Индекс МосБиржи,Индекс МосБиржи,1451.54,1423.40,1455.28,1421.45,4.579322e+10,NaN,NaN,2,6.470954e+12,RUB,NaN,3,None
6,SNDX,IMOEX,2013-09-10,Индекс МосБиржи,Индекс МосБиржи,1453.57,1451.54,1464.20,1446.97,4.892904e+10,NaN,NaN,2,6.480001e+12,RUB,NaN,3,None
7,SNDX,IMOEX,2013-09-11,Индекс МосБиржи,Индекс МосБиржи,1454.12,1452.52,1459.68,1449.58,3.068467e+10,NaN,NaN,2,6.482454e+12,RUB,NaN,3,None
8,SNDX,IMOEX,2013-09-12,Индекс МосБиржи,Индекс МосБиржи,1450.23,1454.21,1461.20,1441.58,3.273912e+10,NaN,NaN,2,6.465093e+12,RUB,NaN,3,None
9,SNDX,IMOEX,2013-09-13,Индекс МосБиржи,Индекс МосБиржи,1440.74,1450.66,1451.89,1434.74,2.469273e+10,NaN,NaN,2,6.422781e+12,RUB,NaN,3,None


Эту же функцию реализуем для индекса металлов и добычи - MOEXMM

In [4]:
# для индекса MOEXMM

def get_moexmm(start_date, end_date, page_size=100):
    moexmm_data = []
    count = 0
    while True:
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/MOEXMM.json?from={start_date}&till={end_date}&start={count}&limit={page_size}'
        try:
            response = requests.get(url)
            data = response.json()
            history_data = data['history']['data']
            headings = data['history']['columns']
            if history_data != None:
                moexmm_data.extend(history_data)
                count += len(history_data)
        except:
            return pd.DataFrame(moexmm_data, columns=headings)

        if len(history_data) < page_size:
            break
    return pd.DataFrame(moexmm_data, columns=headings)


page_size = 100
MOEXMM_df_before_EDA = get_moexmm(start_date, end_date, page_size=page_size)
print(MOEXMM_df_before_EDA.shape)
MOEXMM_df_before_EDA.head(10)

(2842, 18)


,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,MOEXMM,2013-09-02,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2109.53,2103.86,2120.72,2100.19,8.886807e+08,NaN,NaN,2,2.345484e+11,RUB,NaN,3,None
1,SNDX,MOEXMM,2013-09-03,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2102.51,2109.79,2123.56,2089.09,1.304529e+09,NaN,NaN,2,2.337673e+11,RUB,NaN,3,None
2,SNDX,MOEXMM,2013-09-04,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2109.74,2101.98,2112.03,2088.95,1.283077e+09,NaN,NaN,2,2.345715e+11,RUB,NaN,3,None
3,SNDX,MOEXMM,2013-09-05,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2161.35,2110.70,2162.06,2104.88,2.779851e+09,NaN,NaN,2,2.403100e+11,RUB,NaN,3,None
4,SNDX,MOEXMM,2013-09-06,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2157.45,2159.63,2199.92,2154.78,3.672785e+09,NaN,NaN,2,2.398759e+11,RUB,NaN,3,None
5,SNDX,MOEXMM,2013-09-09,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2191.94,2164.13,2199.41,2164.13,3.046784e+09,NaN,NaN,2,2.437106e+11,RUB,NaN,3,None
6,SNDX,MOEXMM,2013-09-10,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2234.30,2195.60,2238.43,2195.60,3.415190e+09,NaN,NaN,2,2.484205e+11,RUB,NaN,3,None
7,SNDX,MOEXMM,2013-09-11,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2205.88,2234.30,2238.07,2204.98,2.719825e+09,NaN,NaN,2,2.452612e+11,RUB,NaN,3,None
8,SNDX,MOEXMM,2013-09-12,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2208.34,2209.81,2225.06,2193.81,3.406006e+09,NaN,NaN,2,2.455348e+11,RUB,NaN,3,None
9,SNDX,MOEXMM,2013-09-13,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2179.43,2205.76,2205.76,2170.27,1.961664e+09,NaN,NaN,2,2.423205e+11,RUB,NaN,3,None


## Парсинг данных с сайта ЦБ

Сайт данных ЦБ динамический, поэтому просто использовать html парсинг в случае ежеджевнообновляемых данных не получилось. Однако у ЦБ есть XML: https://cbr.ru/development/SXML/. В случае парсинга валютной пары USDRUB использовался 'Example 2', для золота был использован 'Example 4'

### Парсинг данных инфляции из данных ЦБ ###

In [5]:
# MOEX и ЦБ обладают разными форматами даты, поэтому установим те же даты, только в формате требуемом сайтом ЦБ

start_date = '01.09.2013'
end_date = '01.01.2025'

In [6]:
# проверим есть ли доступ к сайту
url = f'https://cbr.ru/hd_base/infl/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'
response = requests.get(url)
print(response)

<Response [200]>


In [7]:
def get_inflation(start_date, end_date):
    try:
        url = f'https://cbr.ru/hd_base/infl/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'
        response = requests.get(url)
        tree = BeautifulSoup(response.text, 'html.parser')
        table = tree.find('table', {
            'class': 'data'})  # в коде элемента находим, что таблица хранится в теге table, классе - data

        data = []
        for row in table.find_all('tr')[1:]: 
            cells = row.find_all('td')
            date_str = cells[0].text.strip()
            date = pd.to_datetime(date_str, format='%m.%Y') # инфляция рассчитывается ЦБ раз в месяц, поэтому в переделываем формат в %m.%Y
            Key_rate = cells[1].text.strip()
            Inflation = cells[2].text.strip()
            Inflation_goal = cells[3].text.strip()
            data.append([date, Key_rate, Inflation, Inflation_goal])
        df = pd.DataFrame(data, columns=['Date', 'Key_rate', 'Inflation', 'Inflation_goal'])
        return df

    except Exception as e:
        return f"Ошибка: {e}"


INFLATION_df_before_EDA = get_inflation(start_date, end_date)
print(INFLATION_df_before_EDA.shape)
INFLATION_df_before_EDA.head(10)


(136, 4)


,Date,Key_rate,Inflation,Inflation_goal
0,2024-12-01,"21,00","9,52","4,00"
1,2024-11-01,"21,00","8,88","4,00"
2,2024-10-01,"21,00","8,54","4,00"
3,2024-09-01,"19,00","8,63","4,00"
4,2024-08-01,"18,00","9,05","4,00"
5,2024-07-01,"18,00","9,13","4,00"
6,2024-06-01,"16,00","8,59","4,00"
7,2024-05-01,"16,00","8,30","4,00"
8,2024-04-01,"16,00","7,84","4,00"
9,2024-03-01,"16,00","7,72","4,00"


### Парсинг валютного курса USD RUB из данных ЦБ

In [8]:
# проверили есть ли доступ к сайту
url = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'
response = requests.get(url)
print(response)

<Response [200]>


In [9]:
def get_USDRUB(start_date, end_date):
    try:
        url = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'xml')
        records = soup.find_all('Record')

        data = []
        for record in records:
            i_date = record.get('Date')
            VunitRate = record.find('Value').text.replace(',', '.')

            data.append([i_date, float(VunitRate)])

        df = pd.DataFrame(data, columns=['Date', 'USD_RUB'])
        return df

    except Exception as e:
        return f'Ошибка: \n{e}'


start_date = '01/09/2013'  # здесь потребовался другой формат даты
end_date = '01/01/2025'
USDRUB_df_before_EDA = get_USDRUB(start_date, end_date)
print(USDRUB_df_before_EDA.shape)
USDRUB_df_before_EDA.head(10)

(2800, 2)


,Date,USD_RUB
0,03.09.2013,33.2522
1,04.09.2013,33.3693
2,05.09.2013,33.4656
3,06.09.2013,33.3901
4,07.09.2013,33.4338
5,10.09.2013,33.3243
6,11.09.2013,33.0600
7,12.09.2013,32.9629
8,13.09.2013,32.6731
9,14.09.2013,32.7406


### Парсинг цен на аффинированные драгоценные металлы из данных Банка России ###

In [10]:
# проверили есть ли доступ к сайту
url = f'http://www.cbr.ru/scripts/xml_metall.asp?date_req1={start_date}1&date_req2={end_date}'
response = requests.get(url)
print(response)

<Response [200]>


In [11]:
def get_gold(url):
    try:
        url = f'http://www.cbr.ru/scripts/xml_metall.asp?date_req1={start_date}1&date_req2={end_date}'
        response = requests.get(url)
        tree = BeautifulSoup(response.text, 'html.parser')

        data = []
        for i in tree.find('metall').find_all('record', code='1'):  # ищем элемент <metall>, а затем все элементы record
            i_date = i.get('date')
            buy_price = i.find('buy').text

            data.append([i_date, buy_price])

        df = pd.DataFrame(data, columns=['Date', 'Price_GOLD'])
        return df

    except Exception as e:
        return f'Ошибка \n{e}'


GOLD_df_before_EDA = get_gold(url)
print(GOLD_df_before_EDA .shape)
GOLD_df_before_EDA .head(10)

(2800, 2)


,Date,Price_GOLD
0,03.09.2013,"1487,36"
1,04.09.2013,"1492,6"
2,05.09.2013,"1510,36"
3,06.09.2013,"1494,07"
4,07.09.2013,"1470,76"
5,10.09.2013,"1484,96"
6,11.09.2013,"1459,37"
7,12.09.2013,"1446,87"
8,13.09.2013,"1407,88"
9,14.09.2013,"1377,11"


### Переводим датафреймы в формат CSV

In [12]:
GOLD_df_before_EDA.to_csv('GOLD_df_before_EDA', index=False)
USDRUB_df_before_EDA.to_csv('USDRUB_df_before_EDA', index=False)
INFLATION_df_before_EDA.to_csv('INFLATION_df_before_EDA', index=False)
IMOEX_df_before_EDA.to_csv('IMOEX_df_before_EDA', index=False)
MOEXMM_df_before_EDA.to_csv('MOEXMM_df_before_EDA', index=False)